# Coherence: Comparing levels of explanation

In this notebook we provide a basic implementation for running simulations of the computational-level theories of <span style="font-variant: small-caps;">Coherence</span> and <span style="font-variant: small-caps;">Discriminating Coherence</span>. In addition, there is support for simulating the connectionist algorithmic-level theory for <span style="font-variant: small-caps;">Discriminating Coherence</span>, referred to as ```connectionistDiscriminatingCoherence```.

This notebook makes extensive use of ```mathlib``` for Scala. It is recommended to read [Chapter 9: Scala and mathlib](https://computationalcognitivescience.github.io/lovelace/part_iii/mathlib) for an introduction on this library or to explore the [tutorials](../tutorial/00.00-scala_preface.ipynb).

<div class="alert alert-block alert-warning"><b>Notice.</b> If you are a student in BKI334 Theoretical Modelling for Cognitive Science, please not that this code was developed not specifically for the course. It may need to be modified to suit your own research questions and it may not be bug free.</div>

For completeness here are the computational-level formalizations of <span style="font-variant: small-caps;">Coherence</span> and <span style="font-variant: small-caps;">Discriminating Coherence</span>. We use the truth value assignment variants.

<span style="font-variant: small-caps;">Coherence</span>

*Input:* A graph $G=(V,E)$ with vertex set $V$ and edge set $E\subseteq V\times V$ that partitions into positive constraints $C^+$ and negative constraints $C^-$ (i.e., $C^+\cup C^-=E$ and $C^+\cap C^-=\varnothing$) and a weight function $w: E \rightarrow \mathbb{R}$.

*Output:* A truth value assignemnt $T:V \rightarrow \{true, false\}$ such that $Coh(T)=Coh^+(A,R)+Coh^-(T)$ is maximum. Here, 
$$
Coh^+(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) = T(v)\\
0\text{ otherwise}
\end{cases}
$$
and
$$
Coh^-(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) \ne T(v)\\
0\text{ otherwise}
\end{cases}
$$

<span style="font-variant: small-caps;">Discriminating Coherence</span>

*Input:* A graph $G=(V,E)$ with vertex set $V$ and edge set $E\subseteq V\times V$ that partitions into positive constraints $C^+$ and negative constraints $C^-$ (i.e., $C^+\cup C^-=E$ and $C^+\cap C^-=\varnothing$), a weight function $w: E \rightarrow \mathbb{R}$ and a data weight function $d:D \rightarrow\mathbb{R}$ for favored beliefs $D\subseteq V$.

*Output:* A truth value assignemnt $T:V \rightarrow \{true, false\}$ such that $Coh(T)=Coh^+(T)+Coh^-(T)+Coh^d(T)$ is maximum. Here, 
$$
Coh^+(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) = T(v)\\
0\text{ otherwise}
\end{cases}
$$
and
$$
Coh^-(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) \ne T(v)\\
0\text{ otherwise}
\end{cases}
$$
and
$$
Coh^d(T)=\displaystyle\sum_{v\in D}d(v)
$$

## Importing libraries

The following two code chunks import the necessary libraries for running this notebook. On initial runs you may get messages for downloading ```mathlib```.

In [1]:
import $ivy.`com.markblokpoel::mathlib:0.9.1`
import mathlib.set.SetTheory._

import $ivy.`org.scala-lang.modules::scala-parallel-collections:1.0.4`
import scala.collection.parallel.CollectionConverters._

import $ivy.`com.github.tototoshi::scala-csv:1.3.8`
import java.io.File
import com.github.tototoshi.csv._
import scala.util._

import $ivy.$
import mathlib.set.SetTheory._
import $ivy.$
import scala.collection.parallel.CollectionConverters._
import $ivy.$
import java.io.File
import com.github.tototoshi.csv._
import scala.util._

# Basic Coherence implementation

Below is the implementation of <span style="font-variant: small-caps;">Coherence</span>. 

The function ```def coh(assignment: Map[Vertex, Boolean): Double``` implements $Coh(T)=Coh^+(A,R)+Coh^-(T)$.

The function ```def cohPlus(assignment: Map[Vertex, Boolean]): Double``` implements
$$
Coh^+(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) = T(v)\\
0\text{ otherwise}
\end{cases}
$$

And the function ```def cohMinus(assignment: Map[Vertex, Boolean]): Double``` implements
$$
Coh^-(T)=\displaystyle\sum_{(u,v)\in C^+}
\begin{cases}
w((u,v))\text{ if }T(u) \ne T(v)\\
0\text{ otherwise}
\end{cases}
$$

The output is implemented as an exhaustive search. The following statement returns all possible truth value assignments, i.e., all possible mappings from the set of vertices to the set $\{true, false\}$:
```
vertices.allMappings(Set(true, false))
```
Then from that set find the mapping(s) that have maximal coherence:
```
.argMax(coh _)
```
And if there are more than one optimal solutions, return one at random:
```
.random.get
```

We also add a minimal set of basic datastructures. A vertex is represented by a simple ```String``` and an edge in the network (which could be a positive or negative constraint) is represented by a pair of vertices.

In [2]:
type Vertex = String
type Edge = (Vertex, Vertex)

def coherence(
    vertices: Set[Vertex],
    edges: Set[Edge],
    positiveConstraints: Set[Edge],
    negativeConstraints: Set[Edge]
): Map[Vertex, Boolean] = {
    
    require(positiveConstraints \/ negativeConstraints == edges, "C+ union C- != E")
    require(positiveConstraints /\ negativeConstraints == Set.empty, "C+ intersect C- is not empty")
    
    
    def cohPlus(assignment: Map[Vertex, Boolean]): Int =
     positiveConstraints.count((pc: Edge) => {
        assignment(pc._1) == true && assignment(pc._2) == true ||
        assignment(pc._1) == false && assignment(pc._2) == false
     })
    
    def cohMinus(assignment: Map[Vertex, Boolean]): Int =
     negativeConstraints.count((pc: Edge) => {
        assignment(pc._1) == true && assignment(pc._2) == false ||
        assignment(pc._1) == false && assignment(pc._2) == true
     })
    
    def coh(assignment: Map[Vertex, Boolean]): Int =
        cohPlus(assignment) + cohMinus(assignment)
    
    vertices.allMappings(Set(true, false))
    .argMax(coh _)
    .random.get
}

defined type Vertex
defined type Edge
defined function coherence

## Examples

A simple example belief network to compute an optimal truth value assignment for. Notice that running this code multiple times will lead to different truth value assignments because multiple assignments have maximum coherence.

In [3]:
val elements = Set("a","b","c","d")
val pc = Set(
    ("a","b"),
    ("a","c"),
    ("b","d")
)
val nc = Set(
    ("b","c"),
    ("a","d")
)
coherence(elements, pc \/ nc, pc, nc)

elements: Set[String] = Set("a", "b", "c", "d")
pc: Set[(String, String)] = Set(("a", "b"), ("a", "c"), ("b", "d"))
nc: Set[(String, String)] = Set(("b", "c"), ("a", "d"))
res3_3: Map[Vertex, Boolean] = Map(
  "a" -> false,
  "b" -> true,
  "c" -> false,
  "d" -> true
)

Below is the belief network from [Chapter 5 - Coherece](https://computationalcognitivescience.github.io/lovelace/part_ii/coherence). For convenience, we've added a translation dictionary from belief labels to the descriptions in the network. That way, the output can be printed with nice statements. 

![Coherence example](https://computationalcognitivescience.github.io/lovelace/assets/img/coherence_capital_example.png)

In [4]:
val dict = Map(
    "a" -> "Amsterdam is the largest city",
    "b" -> "Nijmegen is the oldest city",
    "c" -> "The Hague is the capital",
    "d" -> "Amsterdam is the capital",
    "e" -> "Nijmegen is the capital",
    "f" -> "Parliament is in the Hague",
    "g" -> "Parliament is in Amsterdam",
    "h" -> "Parliament is in Nijmegen",
    "i" -> "Prime minister lives in The Hague"
)

val beliefs = Set("a", "b", "c", "d", "e", "f", "g", "h", "i")

val pc = Set(
    ("a","d"),
    ("b","e"),
    ("c","f"),
    ("d","g"),
    ("e","h"),
    ("f","i")
)

val nc = Set(
    ("a","b"),
    ("a","c"),
    ("c","d"),
    ("c","e"),
    ("d","e"),
    ("f","g"),
    ("f","h"),
    ("g","i"),
    ("h","i")
)

val output = coherence(beliefs, pc \/ nc, pc, nc)
output.toSeq.sortBy(_._1).foreach(kv => println(s"${kv._1}\t${dict(kv._1)} is ${kv._2}"))

a	Amsterdam is the largest city is false
b	Nijmegen is the oldest city is true
c	The Hague is the capital is true
d	Amsterdam is the capital is false
e	Nijmegen is the capital is false
f	Parliament is in the Hague is true
g	Parliament is in Amsterdam is false
h	Parliament is in Nijmegen is false
i	Prime minister lives in The Hague is true


dict: Map[String, String] = HashMap(
  "e" -> "Nijmegen is the capital",
  "f" -> "Parliament is in the Hague",
  "a" -> "Amsterdam is the largest city",
  "i" -> "Prime minister lives in The Hague",
  "b" -> "Nijmegen is the oldest city",
  "g" -> "Parliament is in Amsterdam",
  "d" -> "Amsterdam is the capital",
  "c" -> "The Hague is the capital",
  "h" -> "Parliament is in Nijmegen"
)
beliefs: Set[String] = HashSet("e", "f", "a", "i", "b", "g", "d", "c", "h")
pc: Set[(String, String)] = HashSet(
  ("a", "d"),
  ("d", "g"),
  ("e", "h"),
  ("f", "i"),
  ("c", "f"),
  ("b", "e")
)
nc: Set[(String, String)] = HashSet(
  ("a", "b"),
  ("d", "e"),
  ("c", "e"),
  ("g", "i"),
  ("f", "g"),
  ("c", "d"),
  ("h", "i"),
  ("a", "c"),
  ("f", "h")
)
output: Map[Vertex, Boolean] = HashMap(
  "e" -> false,
  "f" -> true,
  "a" -> false,
  "i" -> true,
  "b" -> true,
  "g" -> false,
  "d" -> false,
  "c" -> true,
  "h" -> false
)

# Data Structures

To facilitate writing simulation code, we implement several supporting data structures for representating belief networks. 

## Base Coherence Network

The datastructure ```Network``` builds on basic structures to represent a belief network. In math a belief network $N=(V,E)$ consists of a set of vertices and edges. The set of positive constraints $C^+$ and the set of negative constraints $C^-$ are also represented, and we check if indeed $E=C^+\cup C^-$ as specified in the formal model. Weights for each edge are represented by a ```Map```. This datastructure stores pairs of keys and values, where keys 'point to' the associated value. Here, each edge is a key in the map pointing to a ```Double``` value representing the weight of the edge. For example, the following code would represent two edges and their weights:
```
Map(
  ("a","b") -> 0.4,
  ("a", "c") -> 0.8
)
```
For more examples on the ```Map``` datastructure, see [Section 1.05](..//tutorial/01.05-scala_introduction-collections.ipynb#1.5.4-Maps) of the ```mathlib``` tutorial.

In [5]:
case class Network(
    vertices: Set[Vertex],
    edges: Set[Edge],
    positiveConstraints: Set[Edge],
    negativeConstraints: Set[Edge],
    weights: Map[Edge, Double]
) {
    
    require(positiveConstraints \/ negativeConstraints == edges, "C+ union C- != E")
    require(positiveConstraints /\ negativeConstraints == Set.empty, "C+ intersect C- is not empty")
    
    
    def constraints = positiveConstraints \/ negativeConstraints
    
    def cohPlus(assignment: Map[Vertex, Boolean]): Double = {
        def eval(edge: Edge): Double = {
            if(assignment(edge._1) == assignment(edge._2)) weights(edge)
            else 0.0
        }
        
        positiveConstraints.toList.map(eval).sum
    }
    
    def cohMinus(assignment: Map[Vertex, Boolean]): Double = {
        def eval(edge: Edge): Double = {
            if(assignment(edge._1) != assignment(edge._2)) weights(edge)
            else 0.0
        }
        
        negativeConstraints.toList.map(eval).sum
    }
    
    def coh(assignment: Map[Vertex, Boolean]): Double = {
        cohPlus(assignment) + cohMinus(assignment)
    }
}

defined class Network

### Calculating coherence value

The datastructure contains functions that are equivalent to the implementation of <span style="font-variant: small-caps;">Coherence</span> above that return the coherence value of a given truth value assignment.

Calling the function ```coh``` will return the coherence of the truth value assignment, given the network:

In [6]:
val net = Network(
    vertices = Set("a", "b"),
    edges = Set(("a", "b")),
    positiveConstraints = Set(("a", "b")),
    negativeConstraints = Set.empty,
    weights = Map(("a","b") -> 0.4)
)
val truthValueAssignment = Map("a" -> true, "b" -> true)
net.coh(truthValueAssignment)

net: Network = Network(
  vertices = Set("a", "b"),
  edges = Set(("a", "b")),
  positiveConstraints = Set(("a", "b")),
  negativeConstraints = Set(),
  weights = Map(("a", "b") -> 0.4)
)
truthValueAssignment: Map[String, Boolean] = Map("a" -> true, "b" -> true)
res6_2: Double = 0.4

## Discriminating Coherence Network

For belief networks that favor some beliefs being true over others such as in <span style="font-variant: small-caps;">Discriminating Coherence</span>, we provide an additional datastructure ```DiscriminatingNetwork``` that extends ```Network``` with a weights for the data vertices, also represented by a ```Map```. The set of favored beliefs $D$ is only implicitly represented through the discriminating weights, i.e., if a belief does not have a discriminating weight it is not in $D$.

The coherence value for truth value assignments, relative to a distriminating coherence network can be computed by calling the function ```coh``` on a discriminating network. It implements $Coh(T)=Coh^+(T)+Coh^-(T)+Coh^d(T)$. Here, ```cohPus``` and ```cohMinus``` are inherited from the base coherence network implementation. The function ```def cohD(assignment: Map[Vertex, Boolean]): Double``` implements the coherence bonus for accepting data beliefs:
$$
Coh^d(T)=\displaystyle\sum_{v\in D}d(v)
$$

In [7]:
class DiscriminatingNetwork(
    vertices: Set[Vertex],
    edges: Set[Edge],
    positiveConstraints: Set[Edge],
    negativeConstraints: Set[Edge],
    weights: Map[Edge, Double],
    val dataWeights: Map[Vertex, Double]// Adding val to make value public.
) extends Network(
    vertices: Set[Vertex],
    edges: Set[Edge],
    positiveConstraints: Set[Edge],
    negativeConstraints: Set[Edge],
    weights: Map[Edge, Double]
) {
    def cohD(assignment: Map[Vertex, Boolean]): Double = {
        def eval(vertex: Vertex): Double = {
            if(assignment.get(vertex).getOrElse(false)) dataWeights.get(vertex).getOrElse(0.0)
            else 0.0
        }
        vertices.toList.map(eval).sum
    }
    
    override def coh(assignment: Map[Vertex, Boolean]): Double = {
        cohPlus(assignment) + cohMinus(assignment) + cohD(assignment)
    }
}

case object DiscriminatingNetwork {
    def apply(
        vertices: Set[Vertex],
        edges: Set[Edge],
        positiveConstraints: Set[Edge],
        negativeConstraints: Set[Edge],
        weights: Map[Edge, Double],
        dataWeights: Map[Vertex, Double]
    ): DiscriminatingNetwork = {
        new DiscriminatingNetwork(
            vertices,
            edges: Set[Edge],
            positiveConstraints,
            negativeConstraints,
            weights,
            dataWeights
        )
    }
}

defined class DiscriminatingNetwork
defined object DiscriminatingNetwork

### Calculating discriminating coherence value

Another small example.

In [8]:
val dNet = DiscriminatingNetwork(
    vertices = Set("a", "b"),
    edges = Set(("a", "b")),
    positiveConstraints = Set(("a", "b")),
    negativeConstraints = Set.empty,
    weights = Map(("a","b") -> 0.4),
    dataWeights = Map(("a" -> 0.8))
)
val truthValueAssignment = Map("a" -> true, "b" -> false)
dNet.coh(truthValueAssignment)

dNet: DiscriminatingNetwork = Network(
  vertices = Set("a", "b"),
  edges = Set(("a", "b")),
  positiveConstraints = Set(("a", "b")),
  negativeConstraints = Set(),
  weights = Map(("a", "b") -> 0.4)
)
truthValueAssignment: Map[String, Boolean] = Map("a" -> true, "b" -> false)
res8_2: Double = 0.8

# Discriminating Coherence

The implementation of <span style="font-variant: small-caps;">Discriminating Coherence</span> builds on the ```DiscriminatingNetwork``` data structure. Where otherwise we would list the input of the formalization in full, namely:
```
def discriminatingCoherence(
        vertices: Set[Vertex],
        edges: Set[Edge],
        positiveConstraints: Set[Edge],
        negativeConstraints: Set[Edge],
        weights: Map[Edge, Double],
        dWeights: Map[Vertex, Double]
    ): Map[Vertex, Boolean] = {
}
```
it is here contained in the datastructure. We also do not need to implement the coherence value function, since it is implemented in ```DiscriminatingCoherenceNetwork```.

In [9]:
def discriminatingCoherence(network: DiscriminatingNetwork): Map[Vertex, Boolean] = {
    network.vertices.allMappings(Set(true, false))
    .argMax(network.coh _)
    .random.get
}

defined function discriminatingCoherence

We can now use this implementation to find the truth value assignment with maximal coherence value as follows (reusing ```dNet``` from before):

In [10]:
val dCohOut = discriminatingCoherence(dNet)
val coherenceValue = dNet.coh(dCohOut)

dCohOut: Map[Vertex, Boolean] = Map("a" -> true, "b" -> true)
coherenceValue: Double = 1.2000000000000002

# Connectionist Discriminating Coherence Algorithm

The connectinist coherence algorithm takes a ```discriminatingNetwork``` and transforms it into a connectionnist network. It changes all weights for positive constraints to ```excitatoryWeight```, it changes weights for negative constraints to ```inhibitoryWeight```. It adds a special node ```_s_``` to which all data nodes are connected with weight ```dataWeight```. Node activation initial states are ```initSpecial``` for the special node ```_s_```, ```initOther``` for all other nodes.

The propagation of activations is implemented recursively in the ```step``` subfunction.

In [17]:
def connectionistDCoherence(
    network: DiscriminatingNetwork,
    initSpecial: Double,
    initOther: Double,
    min: Double,
    max: Double,
    decay: Double,
    minChange: Double,
    maxEpochs: Int,
    excitatoryWeight: Double,
    dataWeight: Double,
    inhibitoryWeight: Double
): Map[Vertex, Boolean] = {
    val vertices: Set[Vertex] = network.vertices
    val edges: Set[Edge] = network.edges
    val positiveConstraints: Set[Edge] = network.positiveConstraints
    val negativeConstraints: Set[Edge] = network.negativeConstraints
    val weights: Map[Edge, Double] = network.weights
    val dataWeights: Map[Vertex, Double] = network.dataWeights
    
    val specialVertex: (Vertex, Double) = "_s_" -> initSpecial
    
    def initActivations: Map[Vertex, Double] = {
        val activations = vertices.map(v => {
            v -> initOther
        }).toMap
        
        activations + specialVertex
    }
    
    val weight: Map[Edge, Double] = {
        val specialWeights = dataWeights.keySet.map(d => {
            ("_s_", d) -> dataWeight
        })
        
        val weights = edges.map(e => {
            if(e in positiveConstraints) e -> excitatoryWeight
            else e -> inhibitoryWeight
        }).toMap
        
        weights ++ specialWeights
    }
    
    def net(vertex: Vertex, activations: Map[Vertex, Double]): Double = {
        val n = weight
            .map(ew => {
                val e = ew._1
                val w = ew._2
                if(e._1 == vertex) w * activations(e._2)
                else if(e._2 == vertex) w * activations(e._1)
                else 0.0
            })
            .sum
        math.max(min, math.min(max,n))  // hard limit
        // n / weight.filter(ew => ew._1._1 == vertex || ew._1._2 == vertex).size // normalize by # neighbours as defined in Thagard & Verbeurgt (1998)
    }
    
    def step(activations: Map[Vertex, Double], epoch: Int = 0): Map[Vertex, Double] = {
        val newActivations = vertices.map(vertex => {
            val a_v_t = activations(vertex)
            val n = net(vertex, activations)
            
            val newActivation = a_v_t * (1 - decay) + {
                if(n > 0) n * (max - a_v_t)
                else n * (a_v_t - min)
            }
            
            vertex -> newActivation
        }).toMap + specialVertex
        
        val biggestChange = vertices.toList.map(v => math.abs(activations(v) - newActivations(v))).max
        
        if(epoch > maxEpochs || biggestChange < minChange) newActivations
        else step(newActivations, epoch + 1)
    }
    
    def discretizeOutput(activations: Map[Vertex, Double]): Map[Vertex, Boolean] = {
        activations.view.mapValues(_ > 0).toMap.filter(_._1 != "_s_")
    }
    
    discretizeOutput(step(initActivations))
}

defined function connectionistDCoherence

## Example

We use the Netherlands belief network example to explore the different between the computational-level  and algorithmic-level theory. In principle, we can provide different weights to each individual constraint, but in this example we follow Thagard & Verbeurgt's (1998) parameter settings.

In [18]:
val vertices = Set("a", "b", "c", "d", "e", "f", "g", "h", "i")
val positiveConstraints = Set(
    ("a","d"),
    ("b","e"),
    ("c","f"),
    ("d","g"),
    ("e","h"),
    ("f","i")
)
val negativeConstraints = Set(
    ("a","b"),
    ("a","c"),
    ("c","d"),
    ("c","e"),
    ("d","e"),
    ("f","g"),
    ("f","h"),
    ("g","i"),
    ("h","i")
)
val edges = positiveConstraints \/ negativeConstraints
val weights: Map[Edge, Double] = Map(
    // positive constraints
    ("a","d") -> 0.4,
    ("b","e") -> 0.4,
    ("c","f") -> 0.4,
    ("d","g") -> 0.4,
    ("e","h") -> 0.4,
    ("f","i") -> 0.4,
    // negative constraints
    ("a","b") -> -0.6,
    ("a","c") -> -0.6,
    ("c","d") -> -0.6,
    ("c","e") -> -0.6,
    ("d","e") -> -0.6,
    ("f","g") -> -0.6,
    ("f","h") -> -0.6,
    ("g","i") -> -0.6,
    ("h","i") -> -0.6
)

val dataWeights: Map[Vertex, Double] = Map(
    "a" -> 0.5,
    "c" -> 0.5,
    "d" -> 0.5,
    "g" -> 0.5,
    "i" -> 0.5
)

val dNet = DiscriminatingNetwork(
    vertices,
    edges,
    positiveConstraints,
    negativeConstraints,
    weights,
    dataWeights
)

val discOutputOpt = discriminatingCoherence(dNet)
println("** OPTIMAL NETWORK ***")
dNet.vertices.foreach(bel => println(s"$bel\t${dict(bel)} is ${discOutputOpt(bel)}"))

val neuralOutputOpt = connectionistDCoherence(
    dNet,
    initSpecial = 1.0,
    initOther = 0.1,
    min = -1,
    max = 1,
    decay = 0.05,
    minChange = 0.01,
    maxEpochs = 200,
    excitatoryWeight = 0.4,
    dataWeight = 0.5,
    inhibitoryWeight = -0.6
    
)
println("** NEURAL NETWORK ***")
beliefs.foreach(bel => println(s"$bel\t${dict(bel)} is ${neuralOutputOpt(bel)}"))


println("Belief\tOpt\tNN")
beliefs.toSeq.sorted.foreach(bel => println(s"$bel\t${discOutputOpt(bel)}\t${neuralOutputOpt(bel)}"))


** OPTIMAL NETWORK ***
e	Nijmegen is the capital is true
f	Parliament is in the Hague is true
a	Amsterdam is the largest city is true
i	Prime minister lives in The Hague is true
b	Nijmegen is the oldest city is true
g	Parliament is in Amsterdam is true
d	Amsterdam is the capital is true
c	The Hague is the capital is true
h	Parliament is in Nijmegen is true
** NEURAL NETWORK ***
e	Nijmegen is the capital is false
f	Parliament is in the Hague is true
a	Amsterdam is the largest city is true
i	Prime minister lives in The Hague is true
b	Nijmegen is the oldest city is false
g	Parliament is in Amsterdam is false
d	Amsterdam is the capital is true
c	The Hague is the capital is true
h	Parliament is in Nijmegen is false
Belief	Opt	NN
a	true	true
b	true	false
c	true	true
d	true	true
e	true	false
f	true	true
g	true	false
h	true	false
i	true	true


vertices: Set[String] = HashSet("e", "f", "a", "i", "b", "g", "d", "c", "h")
positiveConstraints: Set[(String, String)] = HashSet(
  ("a", "d"),
  ("d", "g"),
  ("e", "h"),
  ("f", "i"),
  ("c", "f"),
  ("b", "e")
)
negativeConstraints: Set[(String, String)] = HashSet(
  ("a", "b"),
  ("d", "e"),
  ("c", "e"),
  ("g", "i"),
  ("f", "g"),
  ("c", "d"),
  ("h", "i"),
  ("a", "c"),
  ("f", "h")
)
edges: Set[(String, String)] = HashSet(
  ("a", "d"),
  ("a", "b"),
  ("d", "e"),
  ("c", "e"),
  ("g", "i"),
  ("f", "g"),
  ("h", "i"),
  ("d", "g"),
  ("b", "e"),
  ("c", "f"),
  ("c", "d"),
  ("a", "c"),
  ("e", "h"),
  ("f", "i"),
  ("f", "h")
)
weights: Map[Edge, Double] = HashMap(
  ("a", "d") -> 0.4,
  ("a", "b") -> -0.6,
  ("d", "e") -> -0.6,
  ("c", "e") -> -0.6,
  ("g", "i") -> -0.6,
  ("f", "g") -> -0.6,
  ("h", "i") -> -0.6,
  ("d", "g") -> 0.4,
  ("b", "e") -> 0.4,
  ("c", "f") -> 0.4,
  ("c", "d") -> -0.6,
  ("a", "c") -> -0.6,
  ("e", "h") -> 0.4,
  ("f", "i") -> 0.4,
  ("f", "h")

# Generating Random Networks

The object ```RandomNetwork``` provides supporting code for generating random belief networks. It provides limited ways of manipulating properties of the generated networks.


* ```size``` is an integer ```size```$> 0$ and specifies the exact number of vertices that the generated network contains.
* ```density``` is a double $0\leq$ ```density``` $\leq 1$ and specifies the percentage of edges. For example, ```density = 0.4``` means that a random generated network of size $12$ will have $\lfloor 12^2\cdot 0.4\rfloor=57$ edges.
* ```ratioPosNeg``` is a double $0\leq$ ```ratioPosNeg``` $\leq 1$ and specifies the ratio between positive and negative constraints for the edges. For example, ```ratioPosNeg = 0.2``` means that a random generated network with $8$ edges will have $\lfloor 8\cdot 0.2\rfloor=1$ positive constraint and $8-1=7$ negative constraints.
* ```ratioPosNeg``` is a double $0\leq$ ```ratioPosNeg``` $\leq 1$ and specifies the percentage of nodes that are data nodes.

In [24]:
case object RandomNetwork {
    
    def nextDNetwork(
        size: Int,
        density: Double,
        ratioPosNeg: Double,
        dataRatio: Double
    ): DiscriminatingNetwork = {
        val elements = (0 until size)
            .map(i => s"V$i")
            .toSet
        val nrEdges = (density * size).intValue
        val edgeList = Random.shuffle(elements.uniquePairs.toList).take(nrEdges)
        val nrPosConstraints = (ratioPosNeg * edgeList.size).intValue
        val positiveConstraints = edgeList.take(nrPosConstraints).toSet
        val negativeConstraints = edgeList.drop(nrPosConstraints).toSet
        val weights = (positiveConstraints \/ negativeConstraints).map(_ -> Random.nextDouble()).toMap
        val dataWeights = Random.shuffle(elements.toList).take((size * dataRatio).intValue).map(_ -> Random.nextDouble()).toMap
        
        DiscriminatingNetwork(
            vertices = elements,
            edges = positiveConstraints \/ negativeConstraints, 
            positiveConstraints,
            negativeConstraints,
            weights,
            dataWeights
        )
    }
}

defined object RandomNetwork

## Example

In [25]:
RandomNetwork.nextDNetwork(
    size = 8,
    density = 0.7,
    ratioPosNeg = 0.5,
    dataRatio = 0.2
)

res25: DiscriminatingNetwork = Network(
  vertices = HashSet("V4", "V3", "V2", "V7", "V5", "V0", "V1", "V6"),
  edges = HashSet(
    ("V5", "V7"),
    ("V7", "V0"),
    ("V1", "V6"),
    ("V7", "V3"),
    ("V6", "V1")
  ),
  positiveConstraints = Set(("V5", "V7"), ("V1", "V6")),
  negativeConstraints = Set(("V7", "V3"), ("V7", "V0"), ("V6", "V1")),
  weights = HashMap(
    ("V5", "V7") -> 0.08237625218726241,
    ("V7", "V0") -> 0.1581706869307632,
    ("V1", "V6") -> 0.5910136970484955,
    ("V7", "V3") -> 0.2254619153755879,
    ("V6", "V1") -> 0.5190078907361295
  )
)

# Simulating many DCoherence instances

The following code simulates as many DCoherence instances with  <span style="font-variant: small-caps;">Discriminating Coherence</span> and ```connectionistDiscriminatingCoherence```. It computes coherence values for each returned output, relative to the input belief network and it computes the structural difference between the two algortihms' outputs (i.e., how many beliefs are assigned a different truth value).

The second code block below writes the results to a (large) CSV file for analysis. 

In [26]:
// Parameter settings
val size = 12
val density = .8
val ratioPosNeg = .8
val dataRatio = .4
val nrInstances = 1000

// Connectionist parameters
val initSpecial = 1.0
val initOther = 0.1
val min = -1
val max = 1
val decay = 0.05
val minChange = 0.01
val maxEpochs = 10
val excitatoryWeight = 0.4
val dataWeight = -0.6
val inhibitoryWeight = -0.6

// Generate random belief networks
val dBeliefNetworks = for(_ <- 0 until nrInstances) yield
    RandomNetwork.nextDNetwork(size, density, ratioPosNeg, dataRatio)

// Compute all outputs for discriminating and connectionist coherence.
// Computations are parallelized to use multiple CPU cores, remove .par to execute in single core.
val results = dBeliefNetworks.par
    .map(dNetwork => {
        val dOut = discriminatingCoherence(dNetwork)
        val nOut = connectionistDCoherence(
            dNetwork,
            initSpecial,
            initOther,
            min,
            max,
            decay,
            minChange,
            maxEpochs,
            excitatoryWeight,
            dataWeight,
            inhibitoryWeight
        )
        (dNetwork, dOut, dNetwork.coh(dOut), nOut, dNetwork.coh(nOut))
    }).toList

size: Int = 12
density: Double = 0.8
ratioPosNeg: Double = 0.8
dataRatio: Double = 0.4
nrInstances: Int = 1000
initSpecial: Double = 1.0
initOther: Double = 0.1
min: Int = -1
max: Int = 1
decay: Double = 0.05
minChange: Double = 0.01
maxEpochs: Int = 10
excitatoryWeight: Double = 0.4
dataWeight: Double = -0.6
inhibitoryWeight: Double = -0.6
dBeliefNetworks: IndexedSeq[DiscriminatingNetwork] = Vector(
  Network(
    vertices = HashSet(
      "V9",
      "V10",
      "V3",
      "V2",
      "V7",
      "V8",
      "V5",
      "V0",
      "V11",
      "V4",
      "V1",
      "V6"
    ),
    edges = HashSet(
      ("V8", "V3"),
      ("V4", "V7"),
      ("V4", "V11"),
      ("V1", "V6"),
      ("V3", "V1"),
      ("V3", "V5"),
      ("V0", "V9"),
      ("V9", "V2"),
      ("V10", "V8")
    ),
    positiveConstraints = HashSet(
      ("V8", "V3"),
      ("V4", "V7"),
      ("V4", "V11"),
      ("V3", "V1"),
      ("V3", "V5"),
      ("V9", "V2"),
      ("V10", "V8")
    ),
    negativeConst

## CSV file description

We save two CSV files relatable through a random ID number. The first CSV file contains the simulation settings which speak for themselves. The second CSV file contains on each row a single  <span style="font-variant: small-caps;">Discriminating Coherence</span> and ```connectionistDiscriminatingCoherence``` simulation. The following data are stored in columns:

* ```v_1..v_n``` the weights for each discriminated vertex, where regular vertices have weight 0
* ```e_1-1..e_n-n``` the type of constraint, either ```positive```, ```negative``` or ```NA```
* ```w_1-1..w_n-n``` the weight of the constraint, either a double value or ```NA```
* ```opt_v_1..opt_v_n``` the truth value assignment as computed by <span style="font-variant: small-caps;">Discriminating Coherence</span>
* ```opt_coh``` the coherence value of the optimal truth value assignment
* ```opt_v_1..opt_v_n``` the truth value assignment as computed by ```connectionistDiscriminatingCoherence```
* ```opt_coh``` the coherence value of the connectionist truth value assignment
* ```structSim``` the structural similarity between the connectionist truth value assignment and closest optimal truth value assignment

In [27]:
// Save the simulation results to CSV file.
val fileID = Random.nextLong().abs

val fileSettings = new File(s"f$fileID-settings.csv")

val settingsWriter = CSVWriter.open(fileSettings)
settingsWriter.writeRow(
    List(
        "size",
        "density",
        "ratioPosNeg",
        "dataRatio",
        "nrInstances",
        "initSpecial",
        "initOther",
        "min",
        "max",
        "decay",
        "minChange",
        "maxEpochs",
        "excitatoryWeight",
        "dataWeight",
        "inhibitoryWeight"
    )
)
settingsWriter.writeRow(
    List(
        size,
        density,
        ratioPosNeg,
        dataRatio,
        nrInstances,
        initSpecial,
        initOther,
        min,
        max,
        decay,
        minChange,
        maxEpochs,
        excitatoryWeight,
        dataWeight,
        inhibitoryWeight
    )
)

val fileResults = new File(s"f$fileID-results.csv")
val resultsWriter = CSVWriter.open(fileResults)

resultsWriter.writeRow(
    (for(i <- 1 to size) yield s"v_$i") ++
    (for(i <- 1 to size; j <- 1 to size) yield s"e_$i-$j") ++
    (for(i <- 1 to size; j <- 1 to size) yield s"w_$i-$j") ++
    (1 to size).map(i => s"opt_v_$i") ++
    List("opt_coh") ++
    (1 to size).map(i => s"con_v_$i") ++
    List("con_coh", "struct_sim")
)

results.foreach(result => resultsWriter.writeRow({
    val dNet = result._1
    val optOut = result._2
    val optCoh = result._3
    val conOut = result._4
    val conCoh = result._5
    
    val vertices = dNet.vertices.toList.sorted
    
    // datanodes have weights, other nodes have weight 0
    val nodeWeights: List[String] = vertices.map(v => if(dNet.dataWeights.contains(v)) dNet.dataWeights(v).toString else "0")
    // is the edge a positive or negative constraint?
    val edgeTypes: List[String] = for(vi <- vertices; vj <- vertices) yield {
        if(dNet.positiveConstraints.contains((vi, vj)) || dNet.positiveConstraints.contains((vj, vi))) "positive"
        else if(dNet.negativeConstraints.contains((vi, vj)) || dNet.negativeConstraints.contains((vj, vi))) "negative"
        else "NA"
    }
    // the weights of the edges
    val edgeWeights: List[String] = for(vi <- vertices; vj <- vertices) yield {
        if(dNet.weights.contains((vi, vj))) dNet.weights((vi, vj)).toString
        else if(dNet.weights.contains((vj, vi))) dNet.weights((vj, vi)).toString
        else "NA"
    }
    // the optimal coherence output plus coherence value
    val optResults: List[String] = vertices.map(v => optOut(v).toString) ++ List(optCoh.toString)
    // the connectionist coherence output plus coherence value
    val conResults: List[String] = vertices.map(v => conOut(v).toString) ++ List(conCoh.toString)
    // the structural similarity between the connectionist truth value assignment and closest optimal truth value assignment
    val structSim = List({
        val allOptimalSolutions = dNet.vertices.allMappings(Set(true, false)).argMax(dNet.coh _)
        allOptimalSolutions.map(solution => dNet.vertices.filter(v => solution(v) != conOut(v)).size).min
    })
    
    nodeWeights ++ edgeTypes ++ edgeWeights ++ optResults ++ conResults ++ structSim
}))

fileID: Long = 6473862089548275276L
fileSettings: File = f6473862089548275276-settings.csv
settingsWriter: CSVWriter = com.github.tototoshi.csv.CSVWriter@47b8f70f
fileResults: File = f6473862089548275276-results.csv
resultsWriter: CSVWriter = com.github.tototoshi.csv.CSVWriter@d2dfed9